# Natural Language to SQL with OpenAI API
## Enabling IT Managers to Query Support Data Without SQL Knowledge

---

### 📚 What You'll Learn

In this notebook, you'll learn how to build a practical NLP-to-SQL pipeline that converts natural language questions into executable SQL queries. By the end, you'll have a working system that allows non-technical users to query IT support data using plain English.

**End Goal:** Build a system that converts questions like *"How many critical tickets are open?"* into SQL queries, executes them against a database, and returns results in an understandable format.

---

### 🎯 The Business Problem

In IT support organizations, data-driven decision making is critical for operational efficiency and service quality. Managers and stakeholders constantly need to answer important questions about their service desk performance:

- "What's our current ticket backlog?"
- "Which technician is most productive this month?"
- "Are we meeting our SLA targets?"
- "How many critical issues are unresolved?"
- "What categories of problems are most common?"

**The Challenge:** Most managers and stakeholders don't know SQL or how to query databases directly. This creates a significant bottleneck in organizations.

**Current Reality:**

Organizations typically handle this in three ways, all with drawbacks:

1. **Wait for technical staff to run queries** → Slow, creates bottlenecks, technical staff become overwhelmed with reporting requests
2. **Use pre-built dashboards** → Limited flexibility, can't answer ad-hoc questions, doesn't adapt to changing business needs
3. **Export data to Excel manually** → Time-consuming, error-prone, data quickly becomes stale, doesn't scale

These approaches create delays in decision-making, increase workload on technical staff, and prevent agile responses to emerging issues.

---

### 💡 The Solution: NLP-to-SQL Pipeline

Large Language Models (LLMs) have a remarkable capability: they can understand natural language AND generate structured code, including SQL queries. We can leverage this to build an intelligent system that bridges the gap between business users and databases.

**How It Works:**

Our system follows a multi-phase pipeline:

1. **Schema Understanding** → The LLM learns what tables and columns exist in our database
2. **Question Analysis** → A manager asks a question in plain English
3. **SQL Generation** → The LLM converts the question into a valid SQL query
4. **Validation** → We ensure the generated query is safe and correct
5. **Execution** → The query runs against our database
6. **Results Presentation** → Data is returned in an easy-to-understand format


---



This technology has immediate practical applications:

- ✅ **Manager self-service reporting** without requiring SQL knowledge or technical training
- ✅ **Ad-hoc queries during incident reviews** to quickly investigate patterns or anomalies
- ✅ **Quick status checks for stakeholder meetings** to provide up-to-date metrics on demand
- ✅ **Data exploration for process improvement** to identify bottlenecks and optimization opportunities
- ✅ **Training tool for learning SQL patterns** by seeing how natural language maps to queries
- ✅ **Automated reporting workflows** that can be triggered by natural language commands

---

Let's begin building this system step by step!

---

## 🔧 SETUP SECTION

### Step 1: Install Required Dependencies

We need three main libraries:
- **openai** → To communicate with OpenAI's API
- **pandas** → For data manipulation and analysis
- **sqlalchemy** → For database connections and SQL execution

In [ ]:
# Install required packages
!pip install -q openai pandas sqlalchemy

### Step 2: Import Required Libraries

In [ ]:
# Standard library imports
import os
import re
from datetime import datetime, timedelta
from pathlib import Path

# Third-party imports
import pandas as pd
from sqlalchemy import create_engine, text
from openai import OpenAI

print("✅ All libraries imported successfully!")

### Step 3: Configure OpenAI API Key

We'll try to load the API key from Google Colab secrets first (recommended for security), with a fallback to manual input.

In [ ]:
# Configure OpenAI API key
# Method 1: Try to get API key from Colab secrets (recommended)
try:
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    print("✅ API key loaded from Colab secrets")
except:
    # Method 2: Manual input (fallback)
    from getpass import getpass
    print("💡 To use Colab secrets: Go to 🔑 (left sidebar) → Add new secret → Name: OPENAI_API_KEY")
    OPENAI_API_KEY = getpass("Enter your OpenAI API Key: ")

# Set the API key as an environment variable
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# Validate that the API key is set
if not OPENAI_API_KEY or OPENAI_API_KEY.strip() == "":
    raise ValueError("❌ ERROR: No API key provided!")

print("✅ Authentication configured!")

# Configure which OpenAI model to use
OPENAI_MODEL = "gpt-5-nano"  # Using gpt-5-nano for cost efficiency
print(f"🤖 Selected Model: {OPENAI_MODEL}")

# Initialize OpenAI client
client = OpenAI(api_key=OPENAI_API_KEY)
print("✅ OpenAI client initialized!")

---

## 📊 DATA PREPARATION SECTION

### Understanding Our IT Service Desk Data

In IT support environments, we typically track two primary entities: **tickets** (service requests, incidents, and problems) and **technicians** (the staff who resolve these tickets). Understanding this data structure is critical for both humans and the LLM.

The LLM needs to know:
- What tables exist in our database
- What columns each table contains
- What type of data each column holds (text, numbers, dates, etc.)
- What values are typical for each column

This context enables the LLM to generate accurate SQL queries. Without it, the LLM would be guessing at table and column names, leading to errors.

Let's create realistic sample data that mirrors a real IT service desk system.

### Dataset 1: Support Tickets

Our tickets table represents individual support requests from customers.

In [ ]:
# Generate sample IT support ticket data
# Using dates relative to today for realistic time-based queries
today = datetime.now()

tickets_data = [
    # Recent critical tickets (last week)
    {'ticket_id': 'T001', 'title': 'Server outage - production environment down', 'category': 'Network Problem', 'priority': 'Critical', 'status': 'In Progress', 'created_date': (today - timedelta(days=1)).strftime('%Y-%m-%d'), 'resolved_date': None, 'assigned_to': 'Sarah Johnson', 'customer_company': 'SecureBank'},
    {'ticket_id': 'T002', 'title': 'Cannot access shared drive - entire department affected', 'category': 'Access Request', 'priority': 'Critical', 'status': 'Open', 'created_date': (today - timedelta(days=2)).strftime('%Y-%m-%d'), 'resolved_date': None, 'assigned_to': 'Mike Chen', 'customer_company': 'TechCorp'},
    {'ticket_id': 'T003', 'title': 'Email server not responding', 'category': 'Email Issue', 'priority': 'Critical', 'status': 'Resolved', 'created_date': (today - timedelta(days=3)).strftime('%Y-%m-%d'), 'resolved_date': (today - timedelta(days=2)).strftime('%Y-%m-%d'), 'assigned_to': 'Sarah Johnson', 'customer_company': 'DataSystems'},

    # High priority tickets
    {'ticket_id': 'T004', 'title': 'VPN connection keeps dropping', 'category': 'Network Problem', 'priority': 'High', 'status': 'In Progress', 'created_date': (today - timedelta(days=2)).strftime('%Y-%m-%d'), 'resolved_date': None, 'assigned_to': 'Alex Rodriguez', 'customer_company': 'CloudNet'},
    {'ticket_id': 'T005', 'title': 'Laptop won\'t boot - blue screen error', 'category': 'Hardware Issue', 'priority': 'High', 'status': 'Open', 'created_date': (today - timedelta(days=1)).strftime('%Y-%m-%d'), 'resolved_date': None, 'assigned_to': 'John Smith', 'customer_company': 'HealthPlus'},
    {'ticket_id': 'T006', 'title': 'Database connection timeout errors', 'category': 'Software Installation', 'priority': 'High', 'status': 'Resolved', 'created_date': (today - timedelta(days=5)).strftime('%Y-%m-%d'), 'resolved_date': (today - timedelta(days=3)).strftime('%Y-%m-%d'), 'assigned_to': 'Emma Davis', 'customer_company': 'TechCorp'},

    # Medium priority tickets
    {'ticket_id': 'T007', 'title': 'Need admin rights for new software installation', 'category': 'Access Request', 'priority': 'Medium', 'status': 'Open', 'created_date': (today - timedelta(days=3)).strftime('%Y-%m-%d'), 'resolved_date': None, 'assigned_to': 'John Smith', 'customer_company': 'DataSystems'},
    {'ticket_id': 'T008', 'title': 'Email not syncing to mobile device', 'category': 'Email Issue', 'priority': 'Medium', 'status': 'In Progress', 'created_date': (today - timedelta(days=4)).strftime('%Y-%m-%d'), 'resolved_date': None, 'assigned_to': 'Mike Chen', 'customer_company': 'SecureBank'},
    {'ticket_id': 'T009', 'title': 'Printer offline in conference room', 'category': 'Hardware Issue', 'priority': 'Medium', 'status': 'Resolved', 'created_date': (today - timedelta(days=6)).strftime('%Y-%m-%d'), 'resolved_date': (today - timedelta(days=5)).strftime('%Y-%m-%d'), 'assigned_to': 'John Smith', 'customer_company': 'CloudNet'},
    {'ticket_id': 'T010', 'title': 'Software license expired for Adobe Creative Suite', 'category': 'Software Installation', 'priority': 'Medium', 'status': 'Resolved', 'created_date': (today - timedelta(days=7)).strftime('%Y-%m-%d'), 'resolved_date': (today - timedelta(days=4)).strftime('%Y-%m-%d'), 'assigned_to': 'Emma Davis', 'customer_company': 'HealthPlus'},

    # Low priority tickets
    {'ticket_id': 'T011', 'title': 'Request to change desktop wallpaper policy', 'category': 'Access Request', 'priority': 'Low', 'status': 'Open', 'created_date': (today - timedelta(days=10)).strftime('%Y-%m-%d'), 'resolved_date': None, 'assigned_to': 'Alex Rodriguez', 'customer_company': 'TechCorp'},
    {'ticket_id': 'T012', 'title': 'Keyboard spacebar sticking occasionally', 'category': 'Hardware Issue', 'priority': 'Low', 'status': 'Closed', 'created_date': (today - timedelta(days=15)).strftime('%Y-%m-%d'), 'resolved_date': (today - timedelta(days=12)).strftime('%Y-%m-%d'), 'assigned_to': 'John Smith', 'customer_company': 'DataSystems'},

    # Password reset requests (common category)
    {'ticket_id': 'T013', 'title': 'Password reset - forgot domain password', 'category': 'Password Reset', 'priority': 'Medium', 'status': 'Closed', 'created_date': (today - timedelta(days=1)).strftime('%Y-%m-%d'), 'resolved_date': (today - timedelta(days=1)).strftime('%Y-%m-%d'), 'assigned_to': 'Mike Chen', 'customer_company': 'SecureBank'},
    {'ticket_id': 'T014', 'title': 'Account locked after too many failed login attempts', 'category': 'Password Reset', 'priority': 'High', 'status': 'Closed', 'created_date': (today - timedelta(days=2)).strftime('%Y-%m-%d'), 'resolved_date': (today - timedelta(days=2)).strftime('%Y-%m-%d'), 'assigned_to': 'Sarah Johnson', 'customer_company': 'CloudNet'},
    {'ticket_id': 'T015', 'title': 'Need to reset multi-factor authentication', 'category': 'Password Reset', 'priority': 'Medium', 'status': 'Closed', 'created_date': (today - timedelta(days=8)).strftime('%Y-%m-%d'), 'resolved_date': (today - timedelta(days=7)).strftime('%Y-%m-%d'), 'assigned_to': 'Alex Rodriguez', 'customer_company': 'HealthPlus'},

    # Network issues
    {'ticket_id': 'T016', 'title': 'Slow internet connection in office', 'category': 'Network Problem', 'priority': 'Medium', 'status': 'In Progress', 'created_date': (today - timedelta(days=5)).strftime('%Y-%m-%d'), 'resolved_date': None, 'assigned_to': 'Sarah Johnson', 'customer_company': 'TechCorp'},
    {'ticket_id': 'T017', 'title': 'Cannot connect to Wi-Fi network', 'category': 'Network Problem', 'priority': 'Medium', 'status': 'Resolved', 'created_date': (today - timedelta(days=9)).strftime('%Y-%m-%d'), 'resolved_date': (today - timedelta(days=8)).strftime('%Y-%m-%d'), 'assigned_to': 'Alex Rodriguez', 'customer_company': 'DataSystems'},

    # Software installation requests
    {'ticket_id': 'T018', 'title': 'Install Zoom for remote meetings', 'category': 'Software Installation', 'priority': 'Medium', 'status': 'Closed', 'created_date': (today - timedelta(days=11)).strftime('%Y-%m-%d'), 'resolved_date': (today - timedelta(days=10)).strftime('%Y-%m-%d'), 'assigned_to': 'Emma Davis', 'customer_company': 'SecureBank'},
    {'ticket_id': 'T019', 'title': 'Microsoft Office needs to be updated', 'category': 'Software Installation', 'priority': 'Low', 'status': 'Closed', 'created_date': (today - timedelta(days=14)).strftime('%Y-%m-%d'), 'resolved_date': (today - timedelta(days=13)).strftime('%Y-%m-%d'), 'assigned_to': 'Mike Chen', 'customer_company': 'CloudNet'},
    {'ticket_id': 'T020', 'title': 'Install Python development environment', 'category': 'Software Installation', 'priority': 'Medium', 'status': 'Open', 'created_date': (today - timedelta(days=4)).strftime('%Y-%m-%d'), 'resolved_date': None, 'assigned_to': 'Emma Davis', 'customer_company': 'TechCorp'},

    # Additional variety
    {'ticket_id': 'T021', 'title': 'Laptop running very slow - needs optimization', 'category': 'Hardware Issue', 'priority': 'Medium', 'status': 'In Progress', 'created_date': (today - timedelta(days=6)).strftime('%Y-%m-%d'), 'resolved_date': None, 'assigned_to': 'John Smith', 'customer_company': 'HealthPlus'},
    {'ticket_id': 'T022', 'title': 'External monitor not detected', 'category': 'Hardware Issue', 'priority': 'Medium', 'status': 'Resolved', 'created_date': (today - timedelta(days=12)).strftime('%Y-%m-%d'), 'resolved_date': (today - timedelta(days=11)).strftime('%Y-%m-%d'), 'assigned_to': 'John Smith', 'customer_company': 'DataSystems'},
    {'ticket_id': 'T023', 'title': 'Need access to finance folder on SharePoint', 'category': 'Access Request', 'priority': 'High', 'status': 'Closed', 'created_date': (today - timedelta(days=7)).strftime('%Y-%m-%d'), 'resolved_date': (today - timedelta(days=6)).strftime('%Y-%m-%d'), 'assigned_to': 'Alex Rodriguez', 'customer_company': 'SecureBank'},
    {'ticket_id': 'T024', 'title': 'Email attachments not downloading', 'category': 'Email Issue', 'priority': 'High', 'status': 'Resolved', 'created_date': (today - timedelta(days=8)).strftime('%Y-%m-%d'), 'resolved_date': (today - timedelta(days=7)).strftime('%Y-%m-%d'), 'assigned_to': 'Mike Chen', 'customer_company': 'CloudNet'},
    {'ticket_id': 'T025', 'title': 'Spam emails getting through filter', 'category': 'Email Issue', 'priority': 'Low', 'status': 'Open', 'created_date': (today - timedelta(days=13)).strftime('%Y-%m-%d'), 'resolved_date': None, 'assigned_to': 'Sarah Johnson', 'customer_company': 'HealthPlus'},

    # Recent unassigned tickets
    {'ticket_id': 'T026', 'title': 'New hire needs laptop setup', 'category': 'Hardware Issue', 'priority': 'High', 'status': 'Open', 'created_date': today.strftime('%Y-%m-%d'), 'resolved_date': None, 'assigned_to': None, 'customer_company': 'TechCorp'},
    {'ticket_id': 'T027', 'title': 'Conference room TV not working', 'category': 'Hardware Issue', 'priority': 'Medium', 'status': 'Open', 'created_date': today.strftime('%Y-%m-%d'), 'resolved_date': None, 'assigned_to': None, 'customer_company': 'DataSystems'},
]

# Create DataFrame
tickets_df = pd.DataFrame(tickets_data)

print("📋 SUPPORT TICKETS DATA")
print("=" * 80)
print(f"Total tickets: {len(tickets_df)}")
print(f"Date range: {tickets_df['created_date'].min()} to {tickets_df['created_date'].max()}")
print("\nFirst 10 tickets:")
display(tickets_df.head(10))

print("\n📊 Ticket Statistics:")
print(f"  Status breakdown: {tickets_df['status'].value_counts().to_dict()}")
print(f"  Priority breakdown: {tickets_df['priority'].value_counts().to_dict()}")
print(f"  Category breakdown: {tickets_df['category'].value_counts().to_dict()}")

### Dataset 2: Technicians

Our technicians table represents the IT support staff who handle tickets.

In [ ]:
# Generate technician data
technicians_data = [
    {'tech_id': 1, 'name': 'John Smith', 'specialization': 'Hardware', 'hire_date': '2020-03-15', 'availability': 'Full-time'},
    {'tech_id': 2, 'name': 'Sarah Johnson', 'specialization': 'Network', 'hire_date': '2019-07-22', 'availability': 'Full-time'},
    {'tech_id': 3, 'name': 'Mike Chen', 'specialization': 'Software', 'hire_date': '2021-01-10', 'availability': 'Full-time'},
    {'tech_id': 4, 'name': 'Emma Davis', 'specialization': 'Software', 'hire_date': '2021-09-05', 'availability': 'Full-time'},
    {'tech_id': 5, 'name': 'Alex Rodriguez', 'specialization': 'General', 'hire_date': '2022-05-18', 'availability': 'Part-time'},
]

# Create DataFrame
technicians_df = pd.DataFrame(technicians_data)

print("👥 TECHNICIANS DATA")
print("=" * 80)
print(f"Total technicians: {len(technicians_df)}")
display(technicians_df)

print("\n📊 Technician Statistics:")
print(f"  Specializations: {technicians_df['specialization'].value_counts().to_dict()}")
print(f"  Availability: {technicians_df['availability'].value_counts().to_dict()}")

---

## 🗄️ DATABASE SETUP SECTION


In production environments, we typically use enterprise databases like PostgreSQL, MySQL, or Microsoft SQL Server. However, for learning and demonstration purposes, we'll use SQLite.

The NLP-to-SQL pipeline we build here will work with any SQL database with minimal modifications.

### Create SQLite Database

In [ ]:
# Create in-memory SQLite database
# Note: Using in-memory database for this demo (data lost when kernel restarts)
# For persistence, use: create_engine('sqlite:///service_desk.db')
temp_db = create_engine('sqlite:///:memory:')

print("✅ SQLite database created in memory")

### Load Data into Database

In [ ]:
# Load DataFrames into SQLite database as tables
tickets_df.to_sql('tickets', temp_db, index=False, if_exists='replace')
technicians_df.to_sql('technicians', temp_db, index=False, if_exists='replace')

print("✅ Data loaded into database")
print("  - tickets table created with {} records".format(len(tickets_df)))
print("  - technicians table created with {} records".format(len(technicians_df)))

### Verify Database Contents

In [ ]:
# Query to verify tables exist
with temp_db.connect() as conn:
    result = conn.execute(text("SELECT name FROM sqlite_master WHERE type='table'"))
    tables = result.fetchall()
    print("\n📊 Tables in database:")
    for table in tables:
        print(f"  - {table[0]}")

### Test Database with Simple Query

In [ ]:
# Example: Get first 5 tickets from database
with temp_db.connect() as conn:
    result = conn.execute(text("SELECT ticket_id, title, priority, status FROM tickets LIMIT 5"))
    rows = result.fetchall()

print("\n🔍 Sample Query: First 5 tickets")
print("=" * 80)
for row in rows:
    print(row)

print("\n✅ Database is working correctly!")

---

## 🔄 UNDERSTANDING THE WORKFLOW



#### **Phase 1: Schema Context Building** 🏗️
- Generate a comprehensive description of our database structure


#### **Phase 2: Natural Language Input** 💬
- User asks a question in plain English

#### **Phase 3: SQL Generation** 🤖
- Send the schema context + user question to gpt-5-nano


#### **Phase 4: Query Validation & Cleaning** 🛡️
- Extract just the SQL from the LLM response (remove explanatory text)


#### **Phase 5: Execution** ⚙️
- Execute the validated query against our SQLite database


#### **Phase 6: Results Presentation** 📊
- Display results in a user-friendly format




Now let's implement each phase step by step.

---

## 🏗️ PHASE 1: SCHEMA CONTEXT BUILDING

### Teaching the LLM About Our Database

Large Language Models don't have built-in knowledge of your specific database. They know SQL syntax and general database concepts, but they don't know what tables YOU have or what columns exist in YOUR database.

We must explicitly tell the LLM:
- **What tables we have** → "tickets" and "technicians"
- **What columns each table contains** → column names and their purposes
- **What type of data each column holds** → text, integers, dates, etc.
- **Example values** → Show the LLM what actual data looks like

This is called **schema context** or **table definitions**. The more detailed and accurate our description, the better the SQL generation will be.


### Build Schema Context Function

**What this function does:** This function creates a structured text description of a database table that the LLM can understand. It takes a table name and a pandas DataFrame, then generates a detailed summary including all column names with their data types and a few sample rows showing actual values.

In [ ]:
def generate_table_context(table_name, df, num_samples=3):
    """
    Generate schema context for a single table

    This creates a detailed description of the table structure that the LLM can understand.

    Args:
        table_name: Name of the database table
        df: Pandas DataFrame containing the table data
        num_samples: How many sample rows to include

    Returns:
        String containing formatted table description
    """

    context = f"\nTable: {table_name}\n"
    context += "Columns:\n"

    # Add column information with data types
    for col in df.columns:
        dtype = str(df[col].dtype)
        context += f"  - {col} ({dtype})\n"

    # Add sample values to show the LLM what data looks like
    context += f"\nSample rows (first {num_samples}):\n"
    sample_rows = df.head(num_samples).to_dict('records')
    for i, row in enumerate(sample_rows, 1):
        context += f"  Row {i}: {row}\n"

    return context

print("✅ Schema context function created")

### Generate Schema Context for Both Tables

In [ ]:
# Generate context for tickets table
tickets_context = generate_table_context('tickets', tickets_df, num_samples=3)

# Generate context for technicians table
technicians_context = generate_table_context('technicians', technicians_df, num_samples=3)

# Combine into complete database schema context
database_schema = tickets_context + "\n" + technicians_context

print("📋 DATABASE SCHEMA CONTEXT")
print("=" * 80)
print(database_schema)
print("\n✅ Schema context generated successfully")

### Build Complete System Prompt

In [ ]:
# Create the system prompt that will be sent to the LLM
# This prompt gives the LLM its role, context, and instructions

system_prompt = f"""
You are an expert SQL query generator for an IT Service Desk database.

Your task is to convert natural language questions into valid SQL queries.

DATABASE SCHEMA:
{database_schema}

INSTRUCTIONS:
1. Generate ONLY SELECT queries (no INSERT, UPDATE, DELETE, DROP)
2. Use proper SQL syntax for SQLite
3. Return ONLY the SQL query, nothing else
4. End the query with a semicolon
5. Use appropriate JOINs when data from multiple tables is needed
6. Use COUNT, SUM, AVG, etc. for aggregation questions
7. Use WHERE clauses to filter data based on conditions
8. Use ORDER BY and LIMIT when appropriate

IMPORTANT:
- Return ONLY the SQL query
- Do not include explanations or markdown formatting
- Do not use code blocks (```)
- The query should be executable as-is

EXAMPLE:
Question: "How many open tickets are there?"
Response: SELECT COUNT(*) FROM tickets WHERE status = 'Open';
"""

print("✅ System prompt created")
print("\nThis prompt gives the LLM everything it needs to generate accurate SQL queries for our IT service desk database.")

---

## 🤖 PHASE 2 & 3: NATURAL LANGUAGE INPUT AND SQL GENERATION

### Converting Questions to SQL

Now that the LLM understands our database schema, it can generate SQL queries from natural language questions. The process works like this:

1. We send the **system prompt** (which contains the schema) to establish context
2. We send the **user's question** as a message
3. The LLM analyzes the question and generates appropriate SQL
4. The LLM returns the query as a response


### Build Query Generation Function

This is the core function that connects to OpenAI's API and transforms a natural language question into a SQL query. It sends both the system prompt (containing our database schema) and the user's question to gpt-5-nano, then retrieves the generated SQL. The function uses `temperature=0` to ensure consistent, deterministic outputs (the same question always produces the same query), which is important for reliability. It also includes error handling to catch API failures gracefully.

In [ ]:
def generate_sql_query(question, system_prompt):
    """
    Use OpenAI API to convert natural language to SQL

    Args:
        question: Natural language question from user
        system_prompt: System prompt with database schema

    Returns:
        Generated SQL query as string
    """

    print(f"\n💬 Question: {question}")
    print("🤖 Generating SQL query...")

    try:
        # Call OpenAI API using responses.create
        response = client.responses.create(
            model=OPENAI_MODEL,
            input=f"{system_prompt}\n\nUser Question: {question}"
        )

        # Extract the generated SQL
        sql_query = response.output_text.strip()

        print("✅ SQL query generated")
        return sql_query

    except Exception as e:
        print(f"❌ Error generating SQL: {e}")
        return None

print("✅ Query generation function created")

### Test with First Example

In [ ]:
# Test the SQL generation with a simple question
test_question = "How many critical tickets are currently open?"

generated_sql = generate_sql_query(test_question, system_prompt)

if generated_sql:
    print("\n📝 Generated SQL:")
    print("=" * 80)
    print(generated_sql)
    print("=" * 80)

---

## 🛡️ PHASE 4: QUERY VALIDATION AND CLEANING

### Security and Safety Considerations

Large Language Models can generate code, but we should **never blindly execute it**. This is a critical security principle. Even though we've instructed the LLM to only generate SELECT queries, we must validate this programmatically.

**Security Risks:**

1. **SQL Injection** → Malicious users might try to craft questions that generate harmful queries
   - Example: A question designed to drop tables or access sensitive data
   - We must validate that only safe operations are allowed

2. **Incorrect Queries** → The LLM might generate invalid or malformed SQL
   - Syntax errors could crash the application
   - We need to catch these before execution

3. **Harmful Operations** → We only want SELECT queries, not DELETE/UPDATE/DROP
   - Even accidental data modification could be disastrous
   - Strict validation prevents this

**Our Solution: Two-Step Validation Process**

1. **Extract the SQL query** → Remove any explanatory text the LLM might have included
2. **Validate it's safe** → Ensure it's a SELECT query with no forbidden keywords


### Build SQL Extraction Function

This function acts as a "cleaning service" for the LLM's response. Sometimes the LLM might return extra text like "Here's the query you requested:" or wrap the SQL in markdown code blocks. This function uses regex pattern matching to locate and extract ONLY the actual SQL SELECT statement from the response, stripping away any formatting or explanation. It searches for patterns that match SELECT queries ending with semicolons, ensuring we get clean, executable SQL code ready for validation.

In [ ]:
def extract_sql_query(text):
    """
    Extracts SQL SELECT query from LLM response text

    Handles cases where LLM includes:
    - Explanation before/after the query
    - SQL wrapped in markdown code blocks
    - Multiple queries (takes the first)

    Args:
        text: Raw response from LLM

    Returns:
        Cleaned SQL query string or None if no valid query found
    """

    # Remove markdown code blocks if present
    text = re.sub(r'```sql\n?', '', text)
    text = re.sub(r'```\n?', '', text)

    # Pattern to match SELECT queries
    # Matches: SELECT ... ; (case insensitive, multiline)
    pattern = r'SELECT\s+.*?;'

    match = re.search(pattern, text, re.IGNORECASE | re.DOTALL)

    if match:
        query = match.group(0).strip()
        return query
    else:
        print("⚠️ No valid SELECT query found in response")
        return None

print("✅ SQL extraction function created")

### Build SQL Validation Function

This function performs multiple safety checks on the extracted SQL query before allowing it to execute. First, it verifies the query starts with SELECT (read-only operation). Then it scans for dangerous keywords like DELETE, DROP, UPDATE, INSERT, or ALTER that could modify or destroy data. The function uses regex word boundaries (`\b`) to match these keywords as complete words only - this prevents false positives where "CREATE" might appear as part of column names like "created_date". This function implements the principle of "whitelist validation" - only allowing explicitly safe operations.

In [ ]:
def validate_sql_query(query):
    """
    Validates that SQL query is safe to execute

    Security checks:
    - Must be a SELECT statement
    - Cannot contain DELETE, DROP, UPDATE, INSERT, ALTER
    - Must use only allowed tables (tickets, technicians)

    Args:
        query: SQL query string to validate

    Returns:
        Boolean: True if safe, False otherwise
    """

    if not query:
        return False

    # Convert to uppercase for checking
    query_upper = query.upper()

    # Must start with SELECT
    if not query_upper.strip().startswith('SELECT'):
        print("❌ Query must start with SELECT")
        return False

    # Forbidden operations - check for whole words only using word boundaries
    forbidden_keywords = ['DELETE', 'DROP', 'UPDATE', 'INSERT', 'ALTER', 'CREATE', 'TRUNCATE', 'EXEC', 'EXECUTE']
    for keyword in forbidden_keywords:
        # Use regex with word boundaries to match whole words only
        # This prevents matching "CREATE" in "created_date"
        pattern = r'\b' + keyword + r'\b'
        if re.search(pattern, query_upper):
            print(f"❌ Forbidden keyword detected: {keyword}")
            return False

    # Allowed tables only (basic check)
    allowed_tables = ['tickets', 'technicians']

    print("✅ Query passed validation")
    return True

print("✅ SQL validation function created")

### Build Combined Processing Function

This function combines the previous two steps (extraction and validation) into one convenient workflow. It takes the raw text response from the LLM and runs it through both the cleaning process (extract_sql_query) and the security checks (validate_sql_query). Only if both steps succeed does the function return the validated query. If either step fails, it returns None, preventing any unsafe SQL from reaching the database.

In [ ]:
def process_generated_sql(raw_sql):
    """
    Process raw LLM output: extract and validate SQL query

    Args:
        raw_sql: Raw text from LLM

    Returns:
        Validated SQL query or None if invalid
    """

    print("\n🛡️ Processing and validating SQL...")

    # Step 1: Extract SQL
    query = extract_sql_query(raw_sql)

    if not query:
        return None

    print(f"📝 Extracted query: {query[:100]}{'...' if len(query) > 100 else ''}")

    # Step 2: Validate
    if validate_sql_query(query):
        return query
    else:
        return None

print("✅ Combined processing function created")

### Test the Validation

In [ ]:
# Test with our previously generated SQL
if generated_sql:
    validated_query = process_generated_sql(generated_sql)

    if validated_query:
        print("\n✅ VALIDATION SUCCESSFUL")
        print("Query is safe to execute:")
        print(validated_query)
    else:
        print("\n❌ VALIDATION FAILED")
        print("Query was rejected by security checks")

---

## ⚙️ PHASE 5: QUERY EXECUTION

### Running SQL Against the Database

Now we have a validated, safe SQL query that we're confident is secure to execute. The next phase is to actually run it against our SQLite database and retrieve the results.


### Build Query Execution Function

It uses SQLAlchemy's connection context manager (`with` statement) which automatically handles opening and closing the database connection safely, even if errors occur. The function executes the SQL using `conn.execute(text(query))` and fetches all result rows as tuples. It includes comprehensive error handling to catch any database errors (like syntax issues or non-existent columns) and reports them gracefully instead of crashing.

In [ ]:
def execute_sql_query(query, database_engine):
    """
    Execute SQL query against the database

    Args:
        query: Validated SQL query string
        database_engine: SQLAlchemy engine connected to database

    Returns:
        List of result rows (tuples) or None if error
    """

    print("\n⚙️ Executing query...")

    try:
        # Use context manager for safe connection handling
        with database_engine.connect() as conn:
            # Execute the query
            result = conn.execute(text(query))

            # Fetch all rows
            rows = result.fetchall()

            print(f"✅ Query executed successfully")
            print(f"📊 Returned {len(rows)} row(s)")

            return rows

    except Exception as e:
        print(f"❌ Error executing query: {e}")
        return None

print("✅ Query execution function created")

### Execute Our First Complete Query

In [ ]:
# Execute the validated query we generated earlier
if validated_query:
    results = execute_sql_query(validated_query, temp_db)

    if results is not None:
        print("\n📊 QUERY RESULTS")
        print("=" * 80)
        print(f"Question: {test_question}")
        print(f"SQL Query: {validated_query}")
        print(f"\nResults:")
        for row in results:
            print(f"  {row}")
        print("=" * 80)
        print("\n✅ First complete NLP-to-SQL pipeline execution successful!")

---

## 🎯 PHASE 6: COMPLETE PIPELINE FUNCTION

### Putting It All Together

We've successfully built and tested all the individual pieces of our NLP-to-SQL pipeline:

1. ✅ Schema context generation
2. ✅ SQL query generation with LLM
3. ✅ Query validation and security checks
4. ✅ Query execution against database

Now we'll combine these components into a single, easy-to-use function that handles the entire pipeline automatically. This function will:

- Take a natural language question as input
- Process it through all phases
- Return both the generated SQL and the results
- Handle errors gracefully at each step
- Provide progress feedback (optional verbose mode)

This makes it simple to query the database: just call one function with a question!



In [ ]:
def nlp_to_sql_pipeline(question, verbose=False):
    """
    Complete NLP-to-SQL pipeline

    Takes a natural language question and returns database results.
    Handles all phases: generation, validation, execution.

    Args:
        question: Natural language question (string)
        verbose: If True, show detailed progress. If False, only show minimal output (default: False)

    Returns:
        Tuple: (sql_query, results) or (None, None) if any phase fails
    """

    if verbose:
        print("\n" + "="*80)
        print("🚀 STARTING NLP-TO-SQL PIPELINE")
        print("="*80)

    # Phase 1: Generate SQL query
    if verbose:
        print(f"\n💬 Question: {question}")
        print("🤖 Generating SQL query...")

    try:
        response = client.responses.create(
            model=OPENAI_MODEL,
            input=f"{system_prompt}\n\nUser Question: {question}"
        )
        sql_query = response.output_text.strip()
        if verbose:
            print("✅ SQL query generated")
    except Exception as e:
        if verbose:
            print(f"❌ Error generating SQL: {e}")
        return None, None

    if not sql_query:
        if verbose:
            print("❌ Pipeline failed: Could not generate SQL")
        return None, None

    # Phase 2: Validate and clean SQL
    if verbose:
        print("\n🛡️ Processing and validating SQL...")

    validated_sql = process_generated_sql(sql_query) if verbose else process_generated_sql_quiet(sql_query)
    if not validated_sql:
        if verbose:
            print("❌ Pipeline failed: Query validation failed")
        return None, None

    # Phase 3: Execute query
    if verbose:
        print("\n⚙️ Executing query...")

    try:
        with temp_db.connect() as conn:
            result = conn.execute(text(validated_sql))
            rows = result.fetchall()
            if verbose:
                print(f"✅ Query executed successfully")
                print(f"📊 Returned {len(rows)} row(s)")
    except Exception as e:
        if verbose:
            print(f"❌ Error executing query: {e}")
        return validated_sql, None

    if verbose:
        print("\n✅ PIPELINE COMPLETED SUCCESSFULLY")
        print("="*80)

    return validated_sql, rows

# Helper function for quiet validation
def process_generated_sql_quiet(raw_sql):
    """Process and validate SQL without printing messages"""
    query = extract_sql_query(raw_sql)
    if not query:
        return None
    if validate_sql_query_quiet(query):
        return query
    return None

def validate_sql_query_quiet(query):
    """Validate SQL query without printing messages"""
    if not query:
        return False
    query_upper = query.upper()
    if not query_upper.strip().startswith('SELECT'):
        return False
    forbidden_keywords = ['DELETE', 'DROP', 'UPDATE', 'INSERT', 'ALTER', 'CREATE', 'TRUNCATE', 'EXEC', 'EXECUTE']
    for keyword in forbidden_keywords:
        pattern = r'\b' + keyword + r'\b'
        if re.search(pattern, query_upper):
            return False
    return True

print("✅ Complete pipeline function created")

### Build Results Display Function

This function takes the raw query results (which are just tuples of data) and formats them in a clean, manager-friendly way that's easy to read and understand.

In [ ]:
def display_results(question, query, results):
    """
    Display query results in a clean, manager-friendly format

    Args:
        question: Original natural language question
        query: Generated SQL query (optional, for reference)
        results: Query results (list of tuples)
    """

    print("\n" + "="*80)
    print(f"❓ {question}")
    print("="*80)

    # Format results based on type
    if len(results) == 0:
        print("📊 No results found.")
    elif len(results) == 1 and len(results[0]) == 1:
        # Single value result (like COUNT)
        print(f"📊 Answer: {results[0][0]}")
    else:
        # Multiple rows/columns - display as numbered list
        print(f"📊 Found {len(results)} result(s):\n")
        for i, row in enumerate(results, 1):
            # Format each row more readably
            if len(row) == 1:
                print(f"   {i}. {row[0]}")
            else:
                # For multiple columns, format as key-value style
                formatted_row = " | ".join(str(val) for val in row)
                print(f"   {i}. {formatted_row}")

    print("="*80)

print("✅ Display function created")

---

## 💼 PRACTICAL EXAMPLES SECTION

Now let's test our complete NLP-to-SQL pipeline with various real-world questions that IT managers might ask. These examples demonstrate different SQL operations: counting, filtering, aggregation, joins, and time-based queries.

**Note:** By default, the pipeline runs in "clean mode" showing only the question and answer. If you want to see technical details (SQL queries, validation steps, etc.), add `verbose=True` to the pipeline call:
```python
query, results = nlp_to_sql_pipeline(question, verbose=True)
```

### Example 1: Simple Count Query

**Scenario:** Manager wants to know current workload

In [ ]:
# Question 1: Simple count (clean output)
question = "How many tickets are currently open?"
query, results = nlp_to_sql_pipeline(question)

if query and results:
    display_results(question, query, results)

# If you want to see technical details, uncomment the line below:
# query, results = nlp_to_sql_pipeline(question, verbose=True)

### Example 2: Filtering with Conditions

**Scenario:** Manager needs to know about urgent issues

In [ ]:
# Question 2: Filtering by priority
question = "Show me all critical tickets that are not yet resolved"
query, results = nlp_to_sql_pipeline(question)

if query and results:
    display_results(question, query, results)

### Example 3: Technician Performance

**Scenario:** Manager evaluating team productivity

In [ ]:
# Question 3: Technician workload
question = "How many tickets has each technician resolved?"
query, results = nlp_to_sql_pipeline(question)

if query and results:
    display_results(question, query, results)

### Example 4: Time-Based Analysis

**Scenario:** Manager tracking recent activity

In [ ]:
# Question 4: Recent tickets
question = "Show me tickets created in the last 7 days"
query, results = nlp_to_sql_pipeline(question)

if query and results:
    display_results(question, query, results)

### Example 5: Category Analysis

**Scenario:** Manager identifying common problem types

In [ ]:
# Question 5: Most common categories
question = "What are the top 3 most common ticket categories?"
query, results = nlp_to_sql_pipeline(question)

if query and results:
    display_results(question, query, results)

### Example 6: JOIN Query

**Scenario:** Manager wants to see technician specializations with tickets

**Note:** This demonstrates the LLM's ability to generate JOIN queries when data from multiple tables is needed.

In [ ]:
# Question 6: JOIN query across tables
question = "Show me open tickets along with the specialization of the assigned technician"
query, results = nlp_to_sql_pipeline(question)

if query and results:
    display_results(question, query, results)

---

## 🧪 TESTING & EDGE CASES SECTION



Even with excellent prompts and validation, Large Language Models can make mistakes. It's important to understand the system's boundaries and limitations:


1. **LLMs are probabilistic** → They don't always generate perfect queries
2. **Ambiguous questions** → Some questions need clarification from users
3. **Edge cases** → Unusual questions might produce unexpected SQL
4. **Business logic complexity** → Complex requirements may not translate well



### Run Test Suite

In [ ]:
# Define test questions covering different scenarios
test_questions = [
    # Simple queries
    "How many tickets are open?",
    "Count all critical tickets",

    # Aggregations
    "What's the most common ticket category?",
    "Average tickets per technician",

    # Filtering
    "Show me hardware issues",
    "Which tickets are assigned to Sarah Johnson?",

    # Complex
    "Which technician has resolved the most critical tickets?",
    "Show me tickets that took more than 5 days to resolve",

    # Edge cases
    "Find tickets with no assigned technician",
    "What tickets are from TechCorp?",
]

print("🧪 RUNNING TEST SUITE")
print("="*80)

successful_queries = 0
failed_queries = 0

for i, question in enumerate(test_questions, 1):
    print(f"\n\n--- TEST {i}/{len(test_questions)} ---")
    query, results = nlp_to_sql_pipeline(question)

    if query and results is not None:
        successful_queries += 1
        print(f"✅ SUCCESS: {question}")
        print(f"   SQL: {query}")
        print(f"   Results: {len(results)} rows")
    else:
        failed_queries += 1
        print(f"❌ FAILED: {question}")

print("\n\n" + "="*80)
print("📊 TEST SUITE SUMMARY")
print("="*80)
print(f"✅ Successful: {successful_queries}/{len(test_questions)}")
print(f"❌ Failed: {failed_queries}/{len(test_questions)}")
print(f"📈 Success Rate: {(successful_queries/len(test_questions))*100:.1f}%")
print("="*80)

---

## 🎓 BEST PRACTICES & SECURITY CONSIDERATIONS

### For Production Implementation:

**✅ MUST DO:**

1. **Always validate generated SQL before execution** → Never trust LLM output blindly
2. **Use read-only database connections** → Prevent accidental data modification
3. **Log all queries for auditing** → Track who asked what and when
4. **Set appropriate token limits** → Control API costs (SQL queries are typically short)
5. **Use temperature=0 for consistency** → Deterministic outputs for the same question
6. **Provide comprehensive schema context** → Include sample data and relationships
7. **Implement rate limiting** → Prevent API abuse and cost overruns
8. **Add query timeout limits** → Prevent long-running expensive queries
9. **Test extensively** → Validate behavior across many question types
10. **Monitor API costs** → Track tokens per query and set budget alerts

**⚠️ WARNINGS:**

1. **Never expose this system directly to end users without human review** → At least initially, have technical staff verify queries
2. **Don't use for sensitive production databases** → Start with non-critical data
3. **Be aware of hallucinations** → LLM might generate plausible but incorrect SQL
4. **Don't assume 100% accuracy** → Always have a fallback to manual querying
5. **Watch for prompt injection** → Malicious users might try to manipulate the system



**Remember:** This is a learning prototype. Production deployment requires additional security, monitoring, and testing, but you now have the foundational knowledge to build sophisticated NLP-to-SQL systems!